<a href="https://colab.research.google.com/github/olgOk/CohortProject_2020/blob/master/Project_1_RBM_and_Tomography/Train_RNN_Challenge2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib inline

# Added code to run on google colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  ! git clone https://github.com/olgOk/CohortProject_2020.git
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd CohortProject_2020/
  %cd Project_1_RBM_and_Tomography/
  ! git pull

torch.manual_seed(1)

import Rydberg_energy_calculator

# Hyper Parameters
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 1
TIME_STEP = 1          # rnn time step / image height
INPUT_SIZE = 100         # rnn input size / image width
LR = 0.01               # learning rate
DOWNLOAD_MNIST = True   # set to True if haven't download the data

training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt")).float()
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=BATCH_SIZE, shuffle=True)

Cloning into 'CohortProject_2020'...
remote: Enumerating objects: 843, done.
remote: Total 843 (delta 0), reused 0 (delta 0), pack-reused 843
Receiving objects: 100% (843/843), 19.79 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (450/450), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020
/content/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortProject_2020/Project_1_RBM_and_Tomography/CohortPr

In [102]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=INPUT_SIZE,
            hidden_size=64,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(64, 10)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out

In [103]:
rnn = RNN()
print(rnn)

def SimpleLoss(rnn_output, label) :
  return rnn_output + label

optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = SimpleLoss                       



RNN(
  (rnn): LSTM(100, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)


In [105]:
exact_energy = -4.1203519096
n=torch.tensor([-4.1203519096]).long()

# training and testing
for epoch in range(EPOCH):
        if epoch % 100 == 0:
            print("\nEpoch: ", epoch)
            print("Sampling...")
            it = iter(train_loader)
            iter_count = 0
            while iter_count < 1000:
                first = next(it)
                X = first.view(-1, *first.size()) 
                b_first = Variable(first.view(1, 1, 100))              # reshape x to (batch, time_step, input_size)
                RNN_samples = rnn(b_first)                               # rnn output
                iter_count = iter_count + 1
                outp = float(RNN_samples[0][0])
                sampled_energy =  SimpleLoss(outp, exact_energy)  
                if iter_count % 100 == 0:
                  print("\nIteration: ", iter_count)
                  print("Sampling...")                # cross entropy loss
                  print('Energy from RNN sample: ' + str(sampled_energy))
                optimizer.zero_grad()                           # clear gradients for this training step
                #Gradient descent to be improved with cross entropy loss function and backpropagation



Epoch:  0
Sampling...

Iteration:  100
Sampling...
Energy from RNN sample: -4.149465355030994

Iteration:  200
Sampling...
Energy from RNN sample: -4.150871149204473

Iteration:  300
Sampling...
Energy from RNN sample: -4.146119668088655

Iteration:  400
Sampling...
Energy from RNN sample: -4.149465355030994

Iteration:  500
Sampling...
Energy from RNN sample: -4.1589032178146645

Iteration:  600
Sampling...
Energy from RNN sample: -4.15178526093791

Iteration:  700
Sampling...
Energy from RNN sample: -4.1487737399859235

Iteration:  800
Sampling...
Energy from RNN sample: -4.129920273028116

Iteration:  900
Sampling...
Energy from RNN sample: -4.137678111843567

Iteration:  1000
Sampling...
Energy from RNN sample: -4.149465355030994
